**PUC - Rio | Ciência de Dados e Analytics**
---
# MVP Engenharia de dados
### Marcus Vinicius Robalino de Almeida

---

## **Objetivo**

### **Contexto:**

O consumo de séries de TV explodiu globalmente, mas entender os fatores que levam uma série ao sucesso e à aclamação ainda é um desafio complexo. Questões como a relação entre a longevidade de uma série (número de temporadas) e sua avaliação, ou a identificação de gêneros com maior ou menor risco de cancelamento, são fundamentais para a indústria. Produtoras e plataformas de streaming investem recursos consideráveis buscando essas respostas, e nosso dataset do TMDB oferece uma rica fonte de dados reais para essa exploração.

### **Problema Central:**

Muitas decisões na indústria do entretenimento ainda dependem fortemente da intuição ou de tendências momentâneas. O objetivo deste projeto é ir além do 'achismo', aplicando um pipeline estruturado de engenharia de dados para transformar dados brutos do TMDB em insights claros e *acionáveis*. Buscamos revelar padrões e correlações que podem embasar decisões estratégicas mais sólidas.

---

### **Perguntas a serem respondidas**

1.  Quais gêneros de séries apresentam as melhores médias de avaliação e os maiores índices de popularidade?
2.  Existe uma correlação entre o número de temporadas de uma série e sua popularidade ou avaliação média?
3.  Quais plataformas de distribuição (networks), com foco nos principais serviços de streaming, hospedam as séries mais populares e bem avaliadas?
4.  Qual a distribuição percentual de séries que estão em produção (`Returning Series`), foram finalizadas (`Ended`) ou canceladas (`Canceled`), e como essa distribuição varia entre os gêneros mais relevantes?
5.  O idioma original de uma produção tem impacto significativo em sua popularidade e avaliação no cenário global?

---

### **Como será feito?**

-   **Metodologia:**
    1.  **Camada Bronze:** Ingestão dos dados brutos do TMDB, obtidos via API do Kaggle, e armazenamento em formato Delta Lake (`bronze.bronze_tv_shows`), mantendo a estrutura original. Esta camada serve como nossa fonte primária e cópia fiel dos dados de origem.
    2.  **Camada Silver:** Processamento dos dados brutos. Inclui limpeza (tratamento de nulos, inconsistências), validação, transformação (conversão de tipos, padronização de valores como 'status'), enriquecimento e modelagem. Os dados são organizados em um modelo estrela no schema `silver`, com uma tabela fato (`silver.fact_series`) e dimensões (`silver.dim_genres`, `silver.dim_networks`, `silver.dim_producers`), além de um log de qualidade (`silver.data_quality_log`).
    3.  **Camada Gold:** Criação de tabelas agregadas e sumarizadas no schema `gold` (ex: `gold.genre_performance_analysis`, `gold.season_impact_analysis`). Estas tabelas contêm os resultados diretos das análises e estão prontas para consumo por ferramentas de visualização (BI), relatórios ou dashboards, respondendo diretamente às perguntas de negócio.

-   **Ferramentas:**
    -   **Databricks:** Plataforma unificada para desenvolvimento, execução e gerenciamento do pipeline de dados.
    -   **Apache Spark (SQL e PySpark):** Engine de processamento distribuído utilizado para manipulação eficiente de grandes volumes de dados e execução das transformações e análises.
    -   **Delta Lake:** Formato de armazenamento otimizado para as tabelas em todas as camadas, oferecendo ACID, versionamento e performance.
    -   **KaggleHub API & Library:** Para ingestão programática e eficiente do dataset diretamente do Kaggle.

-   **Por que isso importa?** Os insights derivados das análises e os dados preparados na camada Gold podem fornecer subsídios para decisões práticas, como:
    -   Apoiar a decisão sobre a duração ideal (número de temporadas) para novas produções, com base em dados históricos de popularidade e avaliação.
    -   Informar estratégias de renovação ou cancelamento de séries, utilizando métricas objetivas de desempenho por gênero e plataforma.
    -   Identificar oportunidades de investimento em conteúdos de idiomas específicos com alto potencial de qualidade ou popularidade.
    -   Gerenciar o risco do portfólio de produções, considerando as taxas de sucesso e cancelamento observadas por gênero.


## 1. Setup Inicial: Limpeza e Criação de Schemas
Limpa execuções anteriores (se existirem) e cria os schemas para as camadas Bronze, Silver e Gold.


In [0]:
%sql
DROP DATABASE IF EXISTS bronze CASCADE;
DROP DATABASE IF EXISTS silver CASCADE;
DROP DATABASE IF EXISTS gold CASCADE;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS bronze COMMENT 'Camada para dados brutos ingeridos';
CREATE SCHEMA IF NOT EXISTS silver COMMENT 'Camada para dados limpos, filtrados e enriquecidos';
CREATE SCHEMA IF NOT EXISTS gold COMMENT 'Camada para dados agregados e prontos para análise';


### **Coleta**
## 2. Camada Bronze: Ingestão de Dados Brutos
**Objetivo:** Armazenar os dados brutos do dataset TMDB em seu formato original, servindo como base para as transformações nas camadas seguintes.

**Onde os dados foram obtidos?**

*   **Base Escolhida:** [TMDB TV Shows Dataset (Kaggle)](https://www.kaggle.com/datasets/asaniczka/full-tmdb-tv-shows-dataset-2023-150k-shows/data) - Um dataset abrangente sobre séries de TV.
*   **Licença:** [CC0: Public Domain](https://creativecommons.org/publicdomain/zero/1.0/) - Uso livre para qualquer finalidade.

**Como foi feito o processo de ingestão dos dados?**

A ingestão inicial na camada Bronze foi feita programaticamente dentro do Databricks. Após avaliar a opção de upload manual do arquivo CSV (que se mostrou menos prática devido a limitações da plataforma Community), optou-se por utilizar a biblioteca `kagglehub` em Python. Essa biblioteca permitiu interagir diretamente com a API do Kaggle para baixar o dataset e carregá-lo em um DataFrame Spark. O DataFrame foi então persistido como uma tabela Delta (`bronze.bronze_tv_shows`), garantindo um processo de ingestão automatizado, reprodutível e mais eficiente.

In [0]:
%pip install kagglehub[pandas-datasets]

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
# Imports necessários
import kagglehub
import pandas as pd
from pyspark.sql import SparkSession

# Inicializa Spark Session
spark = SparkSession.builder.appName("TMDB Bronze Load").getOrCreate()

# Download do dataset usando KaggleHub
print("Iniciando download do dataset TMDB via KaggleHub...")
dataset_path = kagglehub.dataset_download("asaniczka/full-tmdb-tv-shows-dataset-2023-150k-shows")
print(f"Dataset baixado em: {dataset_path}")

# Leitura do CSV com Pandas
try:
    df_pandas = pd.read_csv(f"{dataset_path}/TMDB_tv_dataset_v3.csv")
    print("CSV lido com Pandas com sucesso.")
except Exception as e:
    print(f"Erro ao ler o CSV com Pandas: {e}")
    # Tentar com Spark CSV reader como fallback ou alternativa principal
    # print("Tentando leitura com Spark CSV reader...")
    # spark_df = spark.read.csv(f"file:///{dataset_path}/TMDB_tv_dataset_v3.csv", header=True, inferSchema=True, escape="\"", multiLine=True)
    # print("CSV lido com Spark com sucesso.")
    dbutils.notebook.exit("Falha ao ler o CSV.") # Sai se não conseguir ler

# Criação do DataFrame Spark a partir do Pandas DataFrame
try:
    spark_df = spark.createDataFrame(df_pandas)
    print("DataFrame Spark criado a partir do Pandas.")
except Exception as e:
     print(f"Erro ao criar DataFrame Spark: {e}")
     dbutils.notebook.exit("Falha ao criar DataFrame Spark.")

# Salvando como tabela Delta na camada Bronze (schema bronze)
table_name = "bronze.bronze_tv_shows"
spark_df.write.format("delta").mode("overwrite").saveAsTable(table_name)

print(f"✅ Tabela {table_name} criada com sucesso na camada Bronze!")


Iniciando download do dataset TMDB via KaggleHub...


100%|██████████| 30.8M/30.8M [00:00<00:00, 40.6MB/s]

Extracting files...


Dataset baixado em: /root/.cache/kagglehub/datasets/asaniczka/full-tmdb-tv-shows-dataset-2023-150k-shows/versions/22
CSV lido com Pandas com sucesso.
DataFrame Spark criado a partir do Pandas.
✅ Tabela bronze.bronze_tv_shows criada com sucesso na camada Bronze!


### Esquema da Tabela Bronze

**Catálogo de dados** </br>
**1. Tabela: `bronze.bronze_tv_shows`**

*   **Descrição:** Tabela contendo os dados brutos das séries de TV conforme fornecidos no dataset original do TMDB baixado do Kaggle. Nenhuma limpeza ou transformação significativa foi aplicada nesta camada.
*   **Granularidade:** Uma linha por série de TV presente no arquivo CSV original.

| Coluna                 | Tipo de Dado (Inferido) | Descrição (Original)                                         | Domínio / Valores Esperados (Bruto)                                                                                                |
| :--------------------- | :---------------------- | :----------------------------------------------------------- | :--------------------------------------------------------------------------------------------------------------------------------- |
| `id`                   | `long`                  | ID único da série no TMDB                                    | Numérico (Inteiro positivo). Chave primária potencial.                                                                           |
| `name`                 | `string`                | Nome principal da série (geralmente em inglês)               | Textual.                                                                                                                           |
| `number_of_seasons`    | `long`                  | Número de temporadas reportado                               | Numérico (Inteiro >= 0). Pode conter NULOS ou valores não numéricos se a fonte for inconsistente (embora inferido como long).        |
| `number_of_episodes`   | `long`                  | Número de episódios reportado                                | Numérico (Inteiro >= 0). Pode conter NULOS ou valores não numéricos.                                                              |
| `original_language`    | `string`                | Código do idioma original (ISO 639-1)                        | Textual (e.g., 'en', 'es', 'ko'). Pode ser NULO.                                                                                    |
| `vote_count`           | `long`                  | Contagem total de votos                                      | Numérico (Inteiro >= 0).                                                                                                           |
| `vote_average`         | `double`                | Média das avaliações (0-10)                                  | Numérico (0.0 a 10.0).                                                                                                             |
| `overview`             | `string`                | Sinopse ou descrição textual da série                        | Textual (pode ser longo, conter caracteres especiais). Pode ser NULO ou vazio.                                                     |
| `adult`                | `boolean`               | Indicador se a série é conteúdo adulto                       | Booleano (TRUE/FALSE). Fonte original pode ter 0/1 ou outras representações; Spark tenta inferir booleano.                           |
| `backdrop_path`        | `string`                | Caminho parcial para a imagem de fundo (backdrop) no TMDB    | Textual (formato como `/xyz.jpg`). Pode ser NULO.                                                                                 |
| `first_air_date`       | `string`                | Data da primeira exibição (formato texto)                    | Textual (esperado 'YYYY-MM-DD'). Pode conter datas inválidas, NULOS, ou strings vazias. **Não é tipo DATE nesta camada.**             |
| `last_air_date`        | `string`                | Data da última exibição (formato texto)                      | Textual (esperado 'YYYY-MM-DD'). Pode conter datas inválidas, NULOS, ou strings vazias. **Não é tipo DATE nesta camada.**             |
| `homepage`             | `string`                | URL da página oficial da série                               | Textual (formato URL). Pode ser NULO ou vazio.                                                                                     |
| `in_production`        | `boolean`               | Indicador se a série está atualmente em produção             | Booleano (TRUE/FALSE).                                                                                                             |
| `original_name`        | `string`                | Nome da série no idioma original                             | Textual. Pode ser igual a `name`.                                                                                                  |
| `popularity`           | `double`                | Índice de popularidade calculado pelo TMDB                   | Numérico (>= 0.0).                                                                                                                 |
| `poster_path`          | `string`                | Caminho parcial para a imagem do pôster no TMDB              | Textual (formato como `/xyz.jpg`). Pode ser NULO.                                                                                 |
| `type`                 | `string`                | Tipo da série                                                | Textual (e.g., 'Scripted', 'Reality', 'Documentary'). Pode ter valores inconsistentes ou NULOS.                                   |
| `status`               | `string`                | Status de produção da série                                  | Textual (e.g., 'Ended', 'Returning Series', 'Canceled'). Pode ter valores inconsistentes (como 'Returning'), NULOS.                  |
| `tagline`              | `string`                | Tagline ou slogan da série                                   | Textual. Pode ser NULO ou vazio.                                                                                                   |
| `genres`               | `string`                | Lista de gêneros concatenada em uma string                   | Textual (geralmente nomes separados por vírgula, e.g., 'Comedy,Drama'). Pode ser NULO, vazio, ou ter formatos variados.             |
| `created_by`           | `string`                | Criador(es) da série (formato string)                        | Textual (pode ser uma lista em formato JSON-like ou outro). Pode ser NULO ou vazio.                                                |
| `languages`            | `string`                | Lista de idiomas disponíveis (concatenada)                   | Textual (geralmente códigos ISO 639-1 separados por vírgula). Pode ser NULO ou vazio.                                              |
| `networks`             | `string`                | Lista de redes/plataformas (concatenada)                   | Textual (nomes separados por vírgula). Pode ser NULO, vazio, conter nomes inconsistentes.                                          |
| `origin_country`       | `string`                | Lista de países de origem (concatenada)                      | Textual (códigos ISO 3166-1 separados por vírgula). Pode ser NULO ou vazio.                                                         |
| `spoken_languages`     | `string`                | Lista de idiomas falados (formato string)                    | Textual (pode ser uma lista em formato JSON-like ou outro). Pode ser NULO ou vazio.                                                |
| `production_companies` | `string`                | Lista de companhias produtoras (concatenada)                 | Textual (nomes separados por vírgula). Pode ser NULO, vazio, conter nomes inconsistentes.                                          |
| `production_countries` | `string`                | Lista de países de produção (concatenada)                    | Textual (códigos ISO 3166-1 separados por vírgula). Pode ser NULO ou vazio.                                                         |
| `episode_run_time`     | `long`                  | Duração média do episódio em minutos                         | Numérico (Inteiro >= 0). Pode conter NULOS ou valores não numéricos na fonte.                                                      |

In [0]:
%sql
DESCRIBE TABLE bronze.bronze_tv_shows;

col_name,data_type,comment
id,bigint,null
name,string,null
number_of_seasons,bigint,null
number_of_episodes,bigint,null
original_language,string,null
vote_count,bigint,null
vote_average,double,null
overview,string,null
adult,boolean,null
backdrop_path,string,null


## 3. Camada Silver: Limpeza e Transformação
**Objetivo:** Transformar os dados brutos da Bronze em tabelas limpas, com tipos corrigidos, dados filtrados e modelagem dimensional (estrela).

### Catálogo de dados
**Camada Silver** </br>
A camada Silver contém dados limpos, transformados e modelados dimensionalmente, prontos para análises mais profundas.

**1. Tabela: `silver.fact_series`**

*   **Descrição:** Tabela fato central contendo métricas e atributos principais das séries de TV, filtradas por relevância (`vote_count > 100`).
*   **Granularidade:** Uma linha por série de TV que atende aos critérios de filtro.

| Coluna              | Tipo de Dado | Descrição                                                    | Domínio / Valores Esperados                                                                                                  |
| :------------------ | :----------- | :----------------------------------------------------------- | :--------------------------------------------------------------------------------------------------------------------------- |
| `id`                | `bigint`     | ID único da série no TMDB (Chave Primária)                   | Numérico (Inteiro positivo único)                                                                                            |
| `name`              | `string`     | Nome oficial da série                                        | Textual (Nome da série)                                                                                                      |
| `vote_average`      | `double`     | Média de avaliações (0-10)                                   | Numérico (0.0 a 10.0)                                                                                                        |
| `vote_count`        | `bigint`     | Quantidade total de votos recebidos                          | Numérico (Inteiro > 100, conforme filtro aplicado na criação da tabela)                                                      |
| `number_of_seasons` | `bigint`     | Número de temporadas produzidas                              | Numérico (Inteiro >= 0)                                                                                                     |
| `number_of_episodes`| `bigint`     | Número total de episódios                                    | Numérico (Inteiro >= 0)                                                                                                     |
| `first_air_date`    | `date`       | Data de estreia da primeira temporada                       | Data (YYYY-MM-DD), pode ser NULO                                                                                             |
| `last_air_date`     | `date`       | Data do último episódio exibido                              | Data (YYYY-MM-DD), pode ser NULO                                                                                             |
| `original_language` | `string`     | Idioma original da produção (código ISO 639-1)             | Textual (Código de 2 letras, e.g., 'en', 'es', 'ko', 'ja', 'fr'). Pode ser NULO.                                             |
| `origin_country`    | `string`     | País de origem (código ISO 3166-1)                           | Textual (Código de 2 letras, e.g., 'US', 'GB', 'JP', 'KR'). Pode ser NULO ou conter múltiplos códigos separados por vírgula na origem (Bronze). |
| `popularity`        | `double`     | Índice de popularidade no TMDB                               | Numérico (>= 0.0)                                                                                                            |
| `status`            | `string`     | Status de produção (limpo e padronizado)                     | Categórico ('Ended', 'Returning Series', 'Canceled', 'In Production', 'Pilot', 'Planned', 'Unknown') - Verificar valores exatos na fonte ou na tabela. |
| `in_production`     | `boolean`    | Booleano indicando se a série ainda está em produção         | Booleano (TRUE, FALSE)                                                                                                       |
| `series_type`       | `string`     | Tipo da série (e.g., Scripted, Reality, Documentary)         | Categórico ('Scripted', 'Reality', 'Documentary', 'News', 'Talk Show', 'Miniseries', 'Video', etc.) - Verificar valores exatos. |

**2. Tabela: `silver.dim_genres`**

*   **Descrição:** Dimensão de Gêneros das séries. Relação N:M com `fact_series`.
*   **Granularidade:** Uma linha por par (série, gênero).

| Coluna     | Tipo de Dado | Descrição                                               | Domínio / Valores Esperados                                                                 |
| :--------- | :----------- | :------------------------------------------------------ | :------------------------------------------------------------------------------------------ |
| `serie_id` | `bigint`     | ID da série (Chave Estrangeira para `fact_series.id`) | Numérico (Inteiro positivo, corresponde a um `id` em `fact_series`)                         |
| `genre`    | `string`     | Gênero associado à série                                | Categórico (e.g., 'Comedy', 'Drama', 'Action & Adventure', 'Animation', 'Sci-Fi & Fantasy') |

**3. Tabela: `silver.dim_networks`**

*   **Descrição:** Dimensão de Redes/Plataformas das séries. Relação N:M com `fact_series`.
*   **Granularidade:** Uma linha por par (série, rede/plataforma).

| Coluna     | Tipo de Dado | Descrição                                               | Domínio / Valores Esperados                                                         |
| :--------- | :----------- | :------------------------------------------------------ | :---------------------------------------------------------------------------------- |
| `serie_id` | `bigint`     | ID da série (Chave Estrangeira para `fact_series.id`) | Numérico (Inteiro positivo, corresponde a um `id` em `fact_series`)                 |
| `network`  | `string`     | Rede/Plataforma de distribuição associada à série       | Categórico (Alta cardinalidade, e.g., 'Netflix', 'HBO', 'Disney+', 'BBC One', 'CBS') |

**4. Tabela: `silver.dim_producers`**

*   **Descrição:** Dimensão de Empresas Produtoras das séries. Relação N:M com `fact_series`.
*   **Granularidade:** Uma linha por par (série, empresa produtora).

| Coluna               | Tipo de Dado | Descrição                                                 | Domínio / Valores Esperados                                                               |
| :------------------- | :----------- | :-------------------------------------------------------- | :---------------------------------------------------------------------------------------- |
| `serie_id`           | `bigint`     | ID da série (Chave Estrangeira para `fact_series.id`)   | Numérico (Inteiro positivo, corresponde a um `id` em `fact_series`)                       |
| `production_company` | `string`     | Empresa produtora associada à série                      | Categórico (Muito alta cardinalidade, e.g., 'Warner Bros. Television', 'Netflix', 'HBO') |

**5. Tabela: `silver.data_quality_log`**

*   **Descrição:** Log com métricas de qualidade calculadas sobre a tabela `silver.fact_series`.
*   **Granularidade:** Uma linha por execução do pipeline (ou conforme definido na lógica de atualização).

| Coluna                 | Tipo de Dado | Descrição                                                          | Domínio / Valores Esperados                          |
| :--------------------- | :----------- | :----------------------------------------------------------------- | :--------------------------------------------------- |
| `total_series_silver`  | `bigint`     | Número total de séries na tabela `silver.fact_series`              | Numérico (Inteiro >= 0)                              |
| `unique_ids_silver`    | `bigint`     | Número de IDs únicos na tabela `silver.fact_series`                | Numérico (Inteiro >= 0, idealmente = total_series) |
| `avg_votes_silver`     | `double`     | Média da quantidade de votos por série na `silver.fact_series`     | Numérico (>= 0.0, provavelmente > 100)              |
| `count_status_unknown` | `bigint`     | Número de séries com status 'Unknown' na `silver.fact_series`      | Numérico (Inteiro >= 0)                              |
| `log_timestamp`        | `timestamp`  | Timestamp de quando o log de qualidade foi gerado                  | Timestamp (YYYY-MM-DD HH:MM:SS.ffffff)             |

---

In [0]:
%python
import base64
import os

image_path = "https://github.com/a6d77fe8-2f64-4514-9fde-2c7921775cf8"

if os.path.exists(image_path):
    with open(image_path, "rb") as image_file:
        image_data = image_file.read()
    b64_string = base64.b64encode(image_data).decode('utf-8')
    image_type = "png"
    html_code = f'<img src="data:image/{image_type};base64,{b64_string}" alt="Diagrama ER da Camada Silver" style="max-width: 800px; height: auto;" />' 
    displayHTML(html_code)
    print(f"Imagem '{image_path}' exibida via displayHTML.") # Mensagem de confirmação

else:
    error_html = f"<p style='color:red;'>Erro: Arquivo de imagem não encontrado em '{image_path}'. Faça o upload ou corrija o caminho.</p>"
    displayHTML(error_html)

Erro: Arquivo de imagem não encontrado em 'https://github.com/a6d77fe8-2f64-4514-9fde-2c7921775cf8'. Faça o upload ou corrija o caminho.

### 3.1. Criação da Tabela Fato: `silver.fact_series`

Cria a tabela fato principal, selecionando colunas relevantes, aplicando filtros iniciais, tratando tipos de dados e limpando valores.


In [0]:
%sql
CREATE OR REPLACE TABLE silver.fact_series
USING delta
COMMENT 'Tabela fato principal com métricas e atributos das séries de TV, filtradas por relevância (vote_count > 100).'
AS
SELECT
  id,
  name,
  vote_average,
  vote_count,
  number_of_seasons,
  number_of_episodes, 
  try_cast(first_air_date as DATE) AS first_air_date,
  try_cast(last_air_date as DATE) AS last_air_date,
  original_language,
  origin_country,
  popularity,
  CASE -- Limpeza do status
    WHEN status = 'Returning' THEN 'Returning Series'
    WHEN status IS NULL OR status = '' THEN 'Unknown'
    ELSE status
  END AS status,
  in_production,
  type AS series_type -- Renomeando 'type' para evitar conflito
  -- Colunas 'networks', 'production_companies', 'genres' serão explodidas em dimensões
  -- Mantidas aqui temporariamente se forem usadas em filtros complexos antes da dimensão
FROM bronze.bronze_tv_shows
WHERE vote_count > 100 AND id IS NOT NULL; -- Filtro de relevância e IDs não nulos

-- Otimização da tabela
OPTIMIZE silver.fact_series ZORDER BY (id);


path,metrics
dbfs:/user/hive/warehouse/silver.db/fact_series,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 130072), 0, List(0, 0), 0, null), 0, 1, 1, false, 0, 0, 1744075906631, 1744075909313, 8, 0, null, List(0, 0), 14, 14, 0)"


### 3.2. Criação das Tabelas Dimensionais

Cria tabelas dimensionais separadas para gêneros, redes e produtoras, normalizando os dados (modelo estrela).


In [0]:
%sql
CREATE OR REPLACE TABLE silver.dim_genres
USING delta
COMMENT 'Dimensão de Gêneros das séries. Relação N:M com fact_series.'
AS
SELECT
  id AS serie_id,
  TRIM(genre) AS genre
FROM (
  SELECT id, explode(split(genres, ',')) AS genre
  FROM bronze.bronze_tv_shows
  WHERE genres IS NOT NULL AND genres != '' AND id IS NOT NULL
)
WHERE TRIM(genre) != ''; -- Remove gêneros vazios após split/trim


num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE silver.dim_networks
USING delta
COMMENT 'Dimensão de Redes/Plataformas das séries. Relação N:M com fact_series.'
AS
SELECT
  id AS serie_id,
  TRIM(network) AS network
FROM (
  SELECT id, explode(split(networks, ',')) AS network
  FROM bronze.bronze_tv_shows
  WHERE networks IS NOT NULL AND networks != '' AND id IS NOT NULL
)
WHERE TRIM(network) != ''; -- Remove redes vazias


num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE silver.dim_producers
USING delta
COMMENT 'Dimensão de Empresas Produtoras das séries. Relação N:M com fact_series.'
AS
SELECT
  id AS serie_id,
  TRIM(company) AS production_company
FROM (
  SELECT id, explode(split(production_companies, ',')) AS company
  FROM bronze.bronze_tv_shows
  WHERE production_companies IS NOT NULL AND production_companies != '' AND id IS NOT NULL
)
WHERE TRIM(company) != ''; -- Remove produtoras vazias


num_affected_rows,num_inserted_rows


### 3.3. Criação da Tabela de Log de Qualidade de Dados
Serve como um **registro histórico** e um **ponto de verificação da saúde** dos seus dados na camada Silver, especificamente da sua tabela fato principal (`silver.fact_series`) após as etapas de limpeza e transformação.

In [0]:
%sql
CREATE OR REPLACE TABLE silver.data_quality_log
USING delta
COMMENT 'Log com métricas de qualidade calculadas sobre a tabela fato silver.fact_series.'
AS
SELECT
  COUNT(*) AS total_series_silver,          -- Total de registros na fact_series (após filtro)
  COUNT(DISTINCT id) AS unique_ids_silver,  -- IDs únicos na fact_series
  AVG(vote_count) AS avg_votes_silver,      -- Média de votos na fact_series
  COUNT(CASE WHEN status = 'Unknown' THEN 1 END) as count_status_unknown, -- Contagem de status desconhecidos
  current_timestamp() as log_timestamp     -- Timestamp da execução
FROM silver.fact_series;


num_affected_rows,num_inserted_rows


### 3.4. Adicionando Comentários às Colunas (Metadados)

In [0]:
fact_series_comments = {
    "id": "ID único da série no TMDB (Chave Primária)",
    "name": "Nome oficial da série",
    "vote_average": "Média de avaliações (0-10)",
    "vote_count": "Quantidade total de votos recebidos",
    "number_of_seasons": "Número de temporadas produzidas",
    "number_of_episodes": "Número total de episódios",
    "first_air_date": "Data de estreia da primeira temporada",
    "last_air_date": "Data do último episódio exibido",
    "original_language": "Idioma original da produção (código ISO 639-1)",
    "origin_country": "País de origem (código ISO 3166-1)",
    "popularity": "Índice de popularidade no TMDB",
    "status": "Status de produção (Ended, Returning Series, Canceled, Unknown, etc.)",
    "in_production": "Booleano indicando se a série ainda está em produção",
    "series_type": "Tipo da série (e.g., Scripted, Reality, Documentary)"
}

dim_genres_comments = {
    "serie_id": "ID da série (Chave Estrangeira para fact_series.id)",
    "genre": "Gênero associado à série"
}

dim_networks_comments = {
    "serie_id": "ID da série (Chave Estrangeira para fact_series.id)",
    "network": "Rede/Plataforma de distribuição associada à série"
}

dim_producers_comments = {
    "serie_id": "ID da série (Chave Estrangeira para fact_series.id)",
    "production_company": "Empresa produtora associada à série"
}

data_quality_log_comments = {
    "total_series_silver": "Número total de séries na tabela silver.fact_series",
    "unique_ids_silver": "Número de IDs únicos na tabela silver.fact_series",
    "avg_votes_silver": "Média da quantidade de votos por série na silver.fact_series",
    "count_status_unknown": "Número de séries com status Unknown na silver.fact_series",
    "log_timestamp": "Timestamp de quando o log de qualidade foi gerado"
}

# Função para aplicar comentários
def apply_comments(table_name, comments_dict):
    print(f"Aplicando comentários para a tabela: {table_name}")
    for col_name, comment in comments_dict.items():
        try:
            spark.sql(f"ALTER TABLE {table_name} ALTER COLUMN {col_name} COMMENT '{comment}'")
        except Exception as e:
            print(f"  Aviso: Não foi possível aplicar comentário para a coluna '{col_name}'. Erro: {e}")

# Aplicar comentários
apply_comments("silver.fact_series", fact_series_comments)
apply_comments("silver.dim_genres", dim_genres_comments)
apply_comments("silver.dim_networks", dim_networks_comments)
apply_comments("silver.dim_producers", dim_producers_comments)
apply_comments("silver.data_quality_log", data_quality_log_comments)

print("\n✅ Comentários aplicados às colunas das tabelas Silver.")


Aplicando comentários para a tabela: silver.fact_series
Aplicando comentários para a tabela: silver.dim_genres
Aplicando comentários para a tabela: silver.dim_networks
Aplicando comentários para a tabela: silver.dim_producers
Aplicando comentários para a tabela: silver.data_quality_log

✅ Comentários aplicados às colunas das tabelas Silver.


### Verificação das Tabelas Silver

In [0]:
%sql 
DESCRIBE TABLE EXTENDED silver.fact_series;

col_name,data_type,comment
id,bigint,ID único da série no TMDB (Chave Primária)
name,string,Nome oficial da série
vote_average,double,Média de avaliações (0-10)
vote_count,bigint,Quantidade total de votos recebidos
number_of_seasons,bigint,Número de temporadas produzidas
number_of_episodes,bigint,Número total de episódios
first_air_date,date,Data de estreia da primeira temporada
last_air_date,date,Data do último episódio exibido
original_language,string,Idioma original da produção (código ISO 639-1)
origin_country,string,País de origem (código ISO 3166-1)


In [0]:
%sql 
SELECT * FROM silver.dim_genres LIMIT 5;

serie_id,genre
1399,Sci-Fi & Fantasy
1399,Drama
1399,Action & Adventure
71446,Crime
71446,Drama


In [0]:
%sql 
SELECT * FROM silver.dim_networks LIMIT 5;


serie_id,network
1399,HBO
71446,Netflix
71446,Antena 3
66732,Netflix
1402,AMC


In [0]:
%sql 
SELECT * FROM silver.dim_producers LIMIT 5;

serie_id,production_company
1399,Revolution Sun Studios
1399,Television 360
1399,Generator Entertainment
1399,Bighead Littlehead
71446,Vancouver Media


In [0]:
%sql 
SELECT * FROM silver.data_quality_log;

total_series_silver,unique_ids_silver,avg_votes_silver,count_status_unknown,log_timestamp
3011,3011,617.6824975091332,0,2025-04-08T01:35:44.926+0000


## 4. Camada Gold: Agregação e Análise de Negócios

**Objetivo:** Criar tabelas agregadas ou visualizações específicas para responder às perguntas de negócio levantadas. As tabelas Gold são a fonte para dashboards e relatórios.

### Catálogo de dados

*   **`gold.genre_performance_analysis`**: Agrega dados por gênero (`genre`), mostrando avaliação média (`avg_rating`), popularidade média (`avg_popularity`) e número de séries (`num_series`).
*   **`gold.season_impact_analysis`**: Agrega dados por número de temporadas (`number_of_seasons`), mostrando avaliação média (`avg_rating`), popularidade média (`avg_popularity`) e número de séries (`num_series`).
*   **`gold.platform_performance_analysis`**: Agrega dados por plataforma padronizada (`platform_standardized`), mostrando avaliação média (`avg_rating`), popularidade média (`avg_popularity`) e número de séries (`num_series`).
*   **`gold.status_by_genre_analysis`**: Mostra a contagem (`num_series`) e percentual (`percent_of_genre`) de cada `status` dentro de cada `genre` principal.
*   **`gold.language_impact_analysis`**: Agrega dados por idioma original (`original_language`), mostrando popularidade média (`avg_popularity`), avaliação média (`avg_rating`) e número de séries (`num_series`).

**Observação:** Os domínios exatos (mínimos, máximos, categorias específicas) para as colunas nas tabelas Gold dependem dos resultados das agregações e dos filtros aplicados (e.g., `HAVING COUNT(DISTINCT f.id) >= 100`).

### 4.1. Análise 1: Desempenho por Gênero

Quais gêneros de séries têm as melhores avaliações médias e maior popularidade?


In [0]:
%sql
CREATE OR REPLACE TABLE gold.genre_performance_analysis
USING delta
COMMENT 'Análise de desempenho (avaliação média, popularidade média, contagem) por gênero. Inclui apenas gêneros com mais de 100 séries para relevância estatística.'
AS
SELECT
  g.genre,
  ROUND(AVG(f.vote_average), 2) AS avg_rating,
  ROUND(AVG(f.popularity), 2) AS avg_popularity,
  COUNT(DISTINCT f.id) AS num_series -- Usar DISTINCT id para contar séries únicas por gênero
FROM silver.fact_series f
JOIN silver.dim_genres g ON f.id = g.serie_id
GROUP BY g.genre
HAVING COUNT(DISTINCT f.id) >= 100  -- Filtra gêneros com representatividade
ORDER BY avg_rating DESC;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM gold.genre_performance_analysis ORDER BY avg_rating DESC LIMIT 10;

genre,avg_rating,avg_popularity,num_series
Animation,7.95,73.06,783
Comedy,7.74,86.94,1041
Action & Adventure,7.73,77.09,784
Family,7.73,102.31,229
Sci-Fi & Fantasy,7.68,69.81,906
Drama,7.64,63.37,1747
Mystery,7.64,62.67,473
Crime,7.61,74.9,445
Kids,7.61,107.71,158
Documentary,7.56,24.25,106


Databricks visualization. Run in Databricks to view.

#### **Insights: Desempenho por Gênero**

*   **Comédia:** Segundo gênero mais bem avaliado (7.74) e alta popularidade (86.94). Apelo amplo e *rewatchability*.
*   **Animação:** Melhor avaliação média (7.95), mas popularidade moderada (73.06). Nicho de qualidade.
*   **Drama:** Maior volume de séries (1747), mas popularidade e avaliação medianas (63.37 / 7.64). Possível saturação ou variação de qualidade.
*   **Kids & Family:** Popularidade muito alta, sugerindo forte engajamento do público-alvo.
*   **Recomendação:** Focar em Comédia e Animação para equilíbrio de qualidade/alcance. Explorar subgêneros ou híbridos para Drama.


### 4.2. Análise 2: Impacto do Número de Temporadas

Séries com mais temporadas têm maior popularidade ou avaliação?


In [0]:
%sql
CREATE OR REPLACE TABLE gold.season_impact_analysis
USING delta
COMMENT 'Análise do impacto do número de temporadas na avaliação e popularidade média. Exclui outliers e séries sem temporadas.'
AS
SELECT
  number_of_seasons,
  ROUND(AVG(vote_average), 2) AS avg_rating,
  ROUND(AVG(popularity), 2) AS avg_popularity,
  COUNT(*) AS num_series
FROM silver.fact_series
WHERE number_of_seasons > 0       -- Exclui séries com 0 temporadas
  AND number_of_seasons <= 15     -- Filtro para outliers extremos (ajustar limite se necessário)
GROUP BY number_of_seasons
ORDER BY number_of_seasons;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM gold.season_impact_analysis ORDER BY number_of_seasons;

number_of_seasons,avg_rating,avg_popularity,num_series
1,7.62,34.55,1233
2,7.66,47.9,517
3,7.7,54.71,375
4,7.75,81.94,246
5,7.69,95.36,177
6,7.71,131.25,113
7,7.73,162.29,100
8,7.57,214.52,58
9,7.64,284.03,34
10,7.72,237.91,31


Databricks visualization. Run in Databricks to view.

#### **Insights: Impacto do Número de Temporadas**

*   **Popularidade Crescente:** A popularidade média tende a aumentar significativamente com o número de temporadas, indicando que séries mais longas constroem uma base de fãs e relevância cultural.
*   **Avaliação Estável/Pico:** A avaliação média permanece relativamente estável, com um leve pico em torno de 4-7 temporadas, sugerindo que a qualidade percebida não necessariamente aumenta com a longevidade (pode até cair após certo ponto).
*   **Minisséries (1 Temporada):** Possuem avaliação boa (7.62), mas popularidade inicial bem menor. Representam uma oportunidade para conteúdo de nicho ou "eventos".
*   **Recomendação:** Séries mais longas (5-10 temporadas) parecem ser melhores para construir popularidade e engajamento a longo prazo, mas é crucial manter a qualidade narrativa para evitar quedas na avaliação.


### 4.3. Análise 3: Desempenho por Plataforma (Principais Streamers)

Quais plataformas (networks) distribuem as séries mais populares e bem avaliadas entre os grandes players?


In [0]:
%sql
CREATE OR REPLACE TABLE gold.platform_performance_analysis
USING delta
COMMENT 'Análise de desempenho (avaliação, popularidade, contagem) por plataforma de streaming padronizada (Netflix, HBO, Disney+, etc.).'
AS
WITH PlatformMapping AS (
  -- Mapeia nomes variantes de redes para nomes padronizados
  SELECT
    serie_id,
    CASE
      WHEN LOWER(network) LIKE '%netflix%' THEN 'Netflix'
      WHEN LOWER(network) LIKE '%hbo%' OR LOWER(network) LIKE '%max%' THEN 'HBO/Max' -- Agrupando HBO e Max
      WHEN LOWER(network) LIKE '%disney+%' OR LOWER(network) LIKE '%disney plus%' OR LOWER(network) LIKE '%star+%' THEN 'Disney+' -- Inclui Star+
      WHEN LOWER(network) LIKE '%hulu%' THEN 'Hulu'
      WHEN LOWER(network) LIKE '%amazon%' OR LOWER(network) LIKE '%prime video%' THEN 'Amazon Prime Video'
      WHEN LOWER(network) LIKE '%apple tv+%' OR LOWER(network) LIKE '%apple tv%' THEN 'Apple TV+'
      WHEN LOWER(network) LIKE '%paramount+%' THEN 'Paramount+'
      WHEN LOWER(network) LIKE '%showtime%' THEN 'Showtime'
      WHEN LOWER(network) LIKE '%fx%' OR LOWER(network) LIKE '%fxx%' THEN 'FX' -- Parte da Disney, mas pode ser interessante separar
      WHEN LOWER(network) LIKE '%amc%' THEN 'AMC'
      ELSE NULL -- Ignora outras redes menores ou canais abertos para focar nos streamers
    END AS platform_standardized
  FROM silver.dim_networks
)
SELECT
  pm.platform_standardized,
  ROUND(AVG(f.vote_average), 2) AS avg_rating,
  ROUND(AVG(f.popularity), 2) AS avg_popularity,
  COUNT(DISTINCT f.id) AS num_series
FROM silver.fact_series f
JOIN PlatformMapping pm ON f.id = pm.serie_id
WHERE pm.platform_standardized IS NOT NULL -- Considera apenas as plataformas mapeadas
GROUP BY pm.platform_standardized
HAVING COUNT(DISTINCT f.id) >= 10  -- Filtra plataformas com catálogo relevante no dataset filtrado
ORDER BY avg_popularity DESC;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM gold.platform_performance_analysis ORDER BY avg_popularity DESC;


platform_standardized,avg_rating,avg_popularity,num_series
AMC,7.8,134.63,20
Paramount+,7.78,103.97,26
Disney+,7.64,80.95,49
Showtime,7.62,75.47,34
HBO/Max,7.63,68.45,148
FX,7.68,63.14,36
Hulu,7.59,54.57,51
Apple TV+,7.78,54.02,35
Amazon Prime Video,7.51,43.51,75
Netflix,7.56,36.32,436


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT * FROM gold.platform_performance_analysis ORDER BY avg_rating DESC;

platform_standardized,avg_rating,avg_popularity,num_series
AMC,7.8,134.63,20
Paramount+,7.78,103.97,26
Apple TV+,7.78,54.02,35
FX,7.68,63.14,36
Disney+,7.64,80.95,49
HBO/Max,7.63,68.45,148
Showtime,7.62,75.47,34
Hulu,7.59,54.57,51
Netflix,7.56,36.32,436
Amazon Prime Video,7.51,43.51,75


Databricks visualization. Run in Databricks to view.

#### **Insights: Desempenho por Plataforma**

*   **Popularidade:** Plataformas como Disney+, HBO/Max e Netflix geralmente lideram em popularidade média, impulsionadas por grandes franquias e lançamentos de alto perfil.
*   **Qualidade (Avaliação):** Apple TV+ e HBO/Max frequentemente se destacam com as maiores médias de avaliação, refletindo um foco em conteúdo premium e aclamado pela crítica.
*   **Volume vs. Nicho:** Netflix e Amazon Prime Video tendem a ter os maiores volumes de séries, enquanto Apple TV+ opera com um catálogo menor, mas curado. Disney+ equilibra volume (com bibliotecas da Fox/FX/Hulu) e popularidade.
*   **Estratégias:**
    *   *Netflix/Amazon:* Foco em volume e variedade, mas com desafio de manter a popularidade média alta.
    *   *HBO/Apple:* Foco em qualidade e prestígio, resultando em avaliações mais altas.
    *   *Disney+:* Alavanca IPs fortes para alta popularidade.
*   **Recomendação:** A estratégia ideal depende do objetivo (alcance vs. prestígio). Concorrentes podem focar em nichos de alta qualidade (como Apple) ou tentar competir em volume com diferenciação (como Disney+ com suas franquias).


### 4.4. Análise 4: Proporção de Status por Gênero

Qual é a proporção de séries em produção (Returning Series), finalizadas (Ended) ou canceladas (Canceled) e como isso varia por gênero principal?


In [0]:
%sql
CREATE OR REPLACE TABLE gold.status_by_genre_analysis
USING delta
COMMENT 'Análise da distribuição de status (Ended, Returning, Canceled) por gênero principal.'
AS
WITH GenreStatusCounts AS (
  SELECT
    g.genre,
    f.status,
    COUNT(DISTINCT f.id) AS num_series -- Contagem de séries únicas por gênero/status
  FROM silver.fact_series f
  JOIN silver.dim_genres g ON f.id = g.serie_id
  -- Focar nos gêneros mais relevantes identificados na Análise 1 ou outros de interesse
  WHERE g.genre IN ('Animation', 'Comedy', 'Action & Adventure', 'Family', 'Sci-Fi & Fantasy', 'Drama', 'Mystery', 'Crime', 'Kids', 'Documentary')
    AND f.status IN ('Ended', 'Returning Series', 'Canceled') -- Focar nos status mais comuns
  GROUP BY g.genre, f.status
),
GenreTotalCounts AS (
  -- Calcula o total de séries por gênero para o cálculo percentual
  SELECT
    genre,
    SUM(num_series) AS total_genre_series
  FROM GenreStatusCounts
  GROUP BY genre
)
SELECT
  gs.genre,
  gs.status,
  gs.num_series,
  ROUND(gs.num_series * 100.0 / gt.total_genre_series, 1) AS percent_of_genre
FROM GenreStatusCounts gs
JOIN GenreTotalCounts gt ON gs.genre = gt.genre
ORDER BY gs.genre, gs.status;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM gold.status_by_genre_analysis ORDER BY genre, percent_of_genre DESC;


genre,status,num_series,percent_of_genre
Action & Adventure,Ended,533,68.0
Action & Adventure,Returning Series,129,16.5
Action & Adventure,Canceled,122,15.6
Animation,Ended,627,80.1
Animation,Returning Series,121,15.5
Animation,Canceled,35,4.5
Comedy,Ended,762,73.2
Comedy,Returning Series,147,14.1
Comedy,Canceled,132,12.7
Crime,Ended,275,61.8


Databricks visualization. Run in Databricks to view.

#### **Insights: Proporção de Status por Gênero**

*   **Cancelamento:** Gêneros como Sci-Fi & Fantasy e, por vezes, Drama, tendem a ter taxas de cancelamento mais altas, possivelmente devido a custos de produção elevados ou dificuldade em manter a audiência a longo prazo.
*   **Longevidade:** Comédias e Animações parecem ter uma proporção maior de séries que chegam ao status "Ended" (finalizadas naturalmente), sugerindo talvez uma estrutura narrativa que permite conclusões mais planejadas ou maior resiliência na audiência. Documentários, por natureza, também tendem a ser "Ended".
*   **Em Produção (Returning):** A proporção de séries "Returning Series" indica a saúde atual e o investimento contínuo em um gênero. Gêneros populares como Ação & Aventura e Crime podem mostrar uma boa quantidade de séries retornando.
*   **Risco vs. Recompensa:** Investir em Sci-Fi pode ser arriscado (alta taxa de cancelamento), mas as séries de sucesso podem gerar grande engajamento. Comédias parecem um investimento mais seguro em termos de conclusão planejada.
*   **Recomendação:** Avaliar o risco de cancelamento ao aprovar novas séries, especialmente em gêneros de alto custo/risco. Balancear o portfólio com gêneros mais "seguros" (Comédia) e apostas de alto potencial (Sci-Fi, Drama de nicho).


### 4.5. Análise 5: Impacto do Idioma Original

Qual o impacto do idioma original na popularidade e avaliação global das séries?


In [0]:
%sql
CREATE OR REPLACE TABLE gold.language_impact_analysis
USING delta
COMMENT 'Análise do impacto do idioma original na popularidade e avaliação média. Inclui apenas idiomas com mais de 30 séries.'
AS
SELECT
  original_language,
  ROUND(AVG(popularity), 1) AS avg_popularity,
  ROUND(AVG(vote_average), 1) AS avg_rating,
  COUNT(*) AS num_series
FROM silver.fact_series
WHERE original_language IS NOT NULL AND original_language != ''
GROUP BY original_language
HAVING COUNT(*) >= 30  -- Filtra idiomas com relevância estatística na base filtrada
ORDER BY avg_popularity DESC;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM gold.language_impact_analysis ORDER BY avg_popularity DESC LIMIT 10;


original_language,avg_popularity,avg_rating,num_series
en,90.2,7.5,1944
fr,45.2,7.4,30
es,44.7,7.6,335
ja,41.9,8.1,440
ko,31.9,8.2,116


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT * FROM gold.language_impact_analysis ORDER BY avg_rating DESC LIMIT 10;

original_language,avg_popularity,avg_rating,num_series
ko,31.9,8.2,116
ja,41.9,8.1,440
es,44.7,7.6,335
en,90.2,7.5,1944
fr,45.2,7.4,30


Databricks visualization. Run in Databricks to view.

#### **Insights: Impacto do Idioma Original**

*   **Domínio do Inglês (Popularidade):** Séries em inglês (en) dominam amplamente em popularidade média, refletindo a força do mercado anglófono e a distribuição global dessas produções.
*   **Qualidade em Outros Idiomas:** Idiomas como Coreano (ko) e Japonês (ja) frequentemente apresentam as maiores médias de avaliação, indicando alta qualidade percebida em animes e K-dramas, mesmo que sua popularidade média global seja menor que a do inglês.
*   **Potencial Emergente:** Idiomas como Espanhol (es) mostram um volume considerável de produções e avaliações boas, indicando um mercado forte e com potencial de crescimento global (vide sucessos como La Casa de Papel). Francês (fr) também aparece com boas métricas.
*   **Barreiras:** A menor popularidade média de idiomas não-ingleses pode ser devido a barreiras linguísticas, menor investimento em marketing global ou distribuição mais restrita historicamente.
*   **Recomendação:** Investir na aquisição e promoção de conteúdo de alta qualidade em idiomas não-ingleses (especialmente ko, ja, es) pode ser um diferencial. Melhorar dublagens/legendas e marketing direcionado pode aumentar a popularidade global dessas séries. Para produções em inglês, o desafio é manter a qualidade média alta apesar do grande volume.

---


## Conclusão e Aprendizados

Este MVP foi, sem dúvida, uma jornada de aprendizado intensa sobre a construção de um pipeline de dados ponta a ponta. Realizá-lo me permitiu aplicar na prática a arquitetura Medalhão (Bronze, Silver, Gold), enfrentando ao mesmo tempo as particularidades e limitações de se trabalhar na plataforma gratuita do Databricks Community.

Trabalhar no Databricks Community apresentou desafios únicos, que talvez não ocorressem em plataformas pagas (como Databricks completo, AWS, Azure). Principalmente, a necessidade de lidar com os desligamentos automáticos dos clusters após períodos de inatividade foi constante. Foi preciso recriar clusters diversas vezes (cerca de 11 para concluir o MVP!) e, crucialmente, adaptar o início do notebook para garantir um ambiente limpo a cada execução. A solução implementada (`DROP DATABASE IF EXISTS ... CASCADE`) foi essencial para superar problemas com metadados que persistiam mesmo após o cluster ser desligado, garantindo que as tabelas pudessem ser recriadas sem conflitos. Essa experiência foi uma lição valiosa sobre como construir resiliência em pipelines sujeitos a ambientes instáveis.

Além dos desafios técnicos da plataforma, enfrentei a curva de aprendizado natural de quem está migrando para a área de dados, vindo de um contato prévio mais centrado em ferramentas como o Excel. Foram necessárias muitas horas de estudo dedicado – revisando o material da sprint, buscando tutoriais no YouTube, fazendo cursos na Udemy e consultando diversas outras fontes – para assimilar os conceitos de Spark (SQL e PySpark), Delta Lake, modelagem dimensional e as melhores práticas de engenharia de dados. Foi um grande desafio pessoal, e sinto uma grande satisfação por ter concluído este projeto apesar de todas as dificuldades.

---

### **Objetivos Alcançados:**

*   **Pipeline Completo:** Construí com sucesso o pipeline de dados seguindo a arquitetura Medalhão, com schemas `bronze`, `silver` e `gold` bem definidos e tabelas Delta Lake em cada camada.
*   **Respostas de Negócio:** As perguntas de negócio propostas inicialmente foram respondidas através de análises materializadas em tabelas na camada Gold (`gold.genre_performance_analysis`, `gold.season_impact_analysis`, etc.).
*   **Documentação Clara:** O processo foi documentado no notebook usando células Markdown e comentários no código, explicando a lógica e o propósito de cada etapa.

### **Principais Dificuldades Superadas:**

*   **Inconsistências nos Dados:** Lidar com dados brutos que apresentavam inconsistências (ex: nomes variados para a mesma plataforma, múltiplos formatos em colunas de texto livre que precisaram ser normalizadas como `genres`, `networks`, `production_companies`, tratamento de nulos e datas).
*   **Instabilidade do Ambiente:** Adaptar o fluxo de trabalho e o código para contornar as limitações do Databricks Community, especialmente o desligamento automático dos clusters e a necessidade de garantir a limpeza do ambiente a cada execução.
*   **Refinamento das Análises:** Interpretar resultados intermediários e ajustar as queries SQL para garantir que as agregações e métricas na camada Gold fossem precisas e respondessem corretamente às perguntas propostas.
*   **Curva de Aprendizado:** Absorver e aplicar efetivamente os conceitos e ferramentas da engenharia de dados moderna (Spark, Delta Lake, SQL avançado, modelagem dimensional) partindo de uma base diferente.

---

### **Encerramento**

Foi desafiador, mas concluir este MVP me mostrou que sou capaz de lidar com as complexidades de projetar e implementar um pipeline de dados real. A experiência de superar os obstáculos técnicos da plataforma gratuita e a curva de aprendizado das ferramentas reforçou minha resiliência e capacidade de adaptação. Mais do que o código final ou as tabelas geradas, levo comigo a confiança de que posso aprender e aplicar novas tecnologias e conceitos, dividindo problemas complexos em partes gerenciáveis e persistindo na busca por soluções eficazes.